# featsql

> Criação de texto para construção de variáveis em janelas de tempo utilizando SQL para os bancos SQLite, MySQL e snowflake. 

As fuções criam variáveis em janelas de tempo fornecidas pelo usuário para variáveis numéricas, categóricas e agregadas. Cada banco tem sua função específica detalhada a seguir:

Para o banco SQLite:

* ``sqlite_create_query_num()``: variáveis numéricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``sqlite_create_query_cat()``: variáveis categóricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``sqlite_create_query_agregada()``: variáveis numéricas agrupadas por uma valor específico de variável categórica com a média, soma e etc na janela de n meses fornecida pelo usuário.

Para o banco MySQL:

* ``mysql_create_query_num()``: variáveis numéricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``mysql_create_query_cat()``: variáveis categóricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``mysql_create_query_agregada()``: variáveis numéricas agrupadas por uma valor específico de variável categórica com a média, soma e etc na janela de n meses fornecida pelo usuário.

Para o banco snowflake:

* ``snow_create_query_num()``: variáveis numéricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``snow_create_query_cat()``: variáveis categóricas como média, soma e etc na janela de n meses fornecida pelo usuário.

* ``snow_create_query_agregada()``: variáveis numéricas agrupadas por uma valor específico de variável categórica com a média, soma e etc na janela de n meses fornecida pelo usuário.

A seguir, temos um exemplo detalhado para a criação de variáveis no banco sqlite e mysql. Além de um exemplo de output para o banco do tipo snowflake.

## Install

```sh
pip install featsql
```

## Imports

In [ ]:
from featsql.featsqlite import *
from featsql.featmysql import *
from featsql.featsnow import *

In [ ]:
import pandas as pd

In [ ]:
#|eval: false

import mysql.connector
import sqlite3
pd.set_option('display.max_columns', None)

# Exemplos

## 1. SQLITE

### Configurando a engine

In [ ]:
#|eval: false
url_db = "../../data/mydatabase.db" 
conn = sqlite3.connect(url_db)
cursor = conn.cursor()


### Visão inicial do público

Primeiro vamos observar o formato da tabela spine

In [ ]:
#|eval: false

df_spine = pd.read_sql("SELECT * FROM tb_spine", conn)
df_spine.head()

,ID,SAFRA_REF,Target
0,4,2023-02-01,0
1,5,2023-02-01,0
2,6,2023-02-01,0
3,7,2023-02-01,0
4,10,2023-02-01,1


### Visão inicial da tabela de variáveis

A tabela de variáveis contém 4 variáveis, duas sendo numéricas e duas categórica. Perceba que existem mais ID's únicos e datas disponíveis nessa tabela do que na tabela spine, caso que ocorre no dia a dia.

In [ ]:
#|eval: false

df_data = pd.read_sql("SELECT * FROM tb_feat", conn)
df_data.head()

,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1,FEAT_CAT2
0,1,2023-01-01,-19,-52,A,B
1,2,2023-01-01,-7,-33,A,B
2,3,2023-01-01,6,91,C,C
3,4,2023-01-01,74,52,B,A
4,5,2023-01-01,79,77,B,B


### Criação de variáveis numéricas

A função sqlite_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
query_final_num_sqlite = sqlite_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_num_sqlite = pd.read_sql(query_final_num_sqlite, conn)
df_num_sqlite.head()

,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0,74,74,74,74.0,52,52,52,52.0,74,74,74,74.0,52,52,52,52.0
1,5,2023-02-01,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0,79,79,79,79.0,77,77,77,77.0,79,79,79,79.0,77,77,77,77.0
2,6,2023-02-01,-13.0,-13.0,-13.0,-13.0,-45.0,-45.0,-45.0,-45.0,-13,-13,-13,-13.0,-45,-45,-45,-45.0,-13,-13,-13,-13.0,-45,-45,-45,-45.0
3,7,2023-02-01,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0,-44,-44,-44,-44.0,0,0,0,0.0,-44,-44,-44,-44.0,0,0,0,0.0
4,10,2023-02-01,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,11,11,11,11.0,-39,-39,-39,-39.0,11,11,11,11.0,-39,-39,-39,-39.0


Podemos utilizar o parâmetro ``nome_arquivo`` com um nome no formato "table.sql" para salvar query de criação em um arquivo na mesma pasta que é executado.

In [ ]:
# |eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
nome_arquivo = 'table.sql'

query_final_num_sqlite = sqlite_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo=nome_arquivo)

Complete query creation with table.sql saved file


Além de salvar a query em um arquivo, podemos criar uma tabela diretamente no banco de dados a partir da função ``sqlite_create_query_num`` com os parâmetros de criação. A seguir um exemplo:

In [ ]:
# |eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query_final_num_sqlite = sqlite_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=conn)

Complete query creation with table.sql saved file


Para conferir a criação, vamos consultar o banco de dados com o nome da tabela que foi criada

In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, conn)
df.head()

,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0
1,5,2023-02-01,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0
2,6,2023-02-01,-13.0,-13.0,-13.0,-13.0,-45.0,-45.0,-45.0,-45.0,-13.0,-13.0,-13.0,-13.0,-45.0,-45.0,-45.0,-45.0,-13.0,-13.0,-13.0,-13.0,-45.0,-45.0,-45.0,-45.0
3,7,2023-02-01,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0
4,10,2023-02-01,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0


### Criação de variáveis categóricas

A função ``sqlite_create_query_cat()`` cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1', 'FEAT_CAT2']
lista_janela = [1, 3, 6]

query_final_cat_sqlite = sqlite_create_query_cat(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_cat_sqlite_sqlite = pd.read_sql(query_final_cat_sqlite, conn)
df_cat_sqlite_sqlite.head()

,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M,FEAT_CAT1_MODA_6M,FEAT_CAT2_MODA_6M
0,4,2023-02-01,B,A,B,A,B,A
1,5,2023-02-01,B,B,B,B,B,B
2,6,2023-02-01,C,B,C,B,C,B
3,7,2023-02-01,B,C,B,C,B,C
4,10,2023-02-01,A,A,A,A,A,A


A função ``sqlite_create_query_cat()`` também permite salvar um arquivo com a query criada e criar a tabela diretamente no banco de dados. Segue um exemplo:

In [ ]:
#|eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1', 'FEAT_CAT2']
lista_janela = [1, 3, 6]
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query_final_cat_sqlite = sqlite_create_query_cat(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=conn)

Complete query creation with table.sql saved file


In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, conn)
df.head()

,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M,FEAT_CAT1_MODA_6M,FEAT_CAT2_MODA_6M
0,4,2023-02-01,B,A,B,A,B,A
1,5,2023-02-01,B,B,B,B,B,B
2,6,2023-02-01,C,B,C,B,C,B
3,7,2023-02-01,B,C,B,C,B,C
4,10,2023-02-01,A,A,A,A,A,A


### Criação de variáveis agregadas

Podemos criar variáveis agregando por valor específico de variável categórica e calculando as operações de sum, avg, min e max para as variáveis númericas definidas pelo usuário. Por exemplo, para a variável ``FEAT_CAT1`` tendo valor igual a ``A`` ou ``B``, calculamos as operações de sum, avg, min e max para as variáveis ``FEAT_NUM1`` e ``FEAT_NUM2``.  Segue um caso de uso:

In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agregador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = sqlite_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agregador)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_sqlite_agregada = pd.read_sql(query, conn)
df_sqlite_agregada.head()

,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0
4,10,2023-02-01,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Segue um exemplo para salvar um arquivo com a query criada e criar a tabela diretamente no banco de dados:

In [ ]:
#|eval: false

tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query = sqlite_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=conn)

Complete query creation with table.sql saved file


Para conferir a criação, vamos consultar o banco de dados com o nome da tabela que foi criada:

In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, conn)
df.head()

,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0,74.0,74.0,74.0,74.0,52.0,52.0,52.0,52.0
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0,79.0,79.0,79.0,79.0,77.0,77.0,77.0,77.0
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0,-44.0,-44.0,-44.0,-44.0,0.0,0.0,0.0,0.0
4,10,2023-02-01,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,11.0,11.0,11.0,11.0,-39.0,-39.0,-39.0,-39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. MySQL

### Configurando conexão 

In [ ]:
#|eval: false

host = "localhost"
user = "sqluser"
password = "password"
database = "mydatabase"

# Conectar ao MySQL
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

### Visão inicial do público

In [ ]:
#|eval: false

df_spine = pd.read_sql("SELECT * FROM tb_spine", connection)
df_spine.head()

/tmp/ipykernel_10461/1385309256.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_spine = pd.read_sql("SELECT * FROM tb_spine", connection)


,ID,SAFRA_REF,Target
0,4,2023-02-01,1
1,5,2023-02-01,0
2,6,2023-02-01,0
3,7,2023-02-01,0
4,10,2023-02-01,0


### Visão inicial da tabela de variáveis

In [ ]:
#|eval: false

df_data = pd.read_sql("SELECT * FROM tb_feat", connection)
df_data.head()

/tmp/ipykernel_10461/548342132.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_data = pd.read_sql("SELECT * FROM tb_feat", connection)


,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1,FEAT_CAT2
0,1,2023-01-01,73,23,B,B
1,3,2023-01-01,15,1,B,B
2,5,2023-01-01,75,71,A,A
3,7,2023-01-01,73,82,B,C
4,9,2023-01-01,61,8,C,B


### Criação de variáveis numéricas

A função mysql_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
query_final_num_mysql = mysql_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_num_mysql = pd.read_sql(query_final_num_mysql, connection)
df_num_mysql.head()

/tmp/ipykernel_10461/3114881597.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num_mysql = pd.read_sql(query_final_num_mysql, connection)


,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A função ``mysql_create_query_num()`` permite a criação de um arquivo no formato .sql da query e também a execução da query direto no banco de dados. Segue um exemplo:

In [ ]:
# |eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query_final_num_mysql = mysql_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=connection)

Complete query creation with table.sql saved file


Para verificar se a tabela foi realmente criada, podemos acessar direto no banco de dados:

In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, connection)
df.head()

/tmp/ipykernel_10461/3805114983.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Criação de variáveis categóricas

A função ``mysql_create_query_cat()`` cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1','FEAT_CAT2']
lista_janela = [1,2,3]
query_final_cat_mysql = mysql_create_query_cat(tb_publico, tb_feat, lista_janela, feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_cat_sqlite_mysql = pd.read_sql(query_final_cat_mysql, connection)
df_cat_sqlite_mysql.head()

/tmp/ipykernel_10461/827211967.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cat_sqlite_mysql = pd.read_sql(query_final_cat_mysql, connection)


,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_2M,FEAT_CAT2_MODA_2M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M
0,4,2023-02-01,None,None,None,None,None,None
1,5,2023-02-01,A,A,A,A,A,A
2,6,2023-02-01,None,None,None,None,None,None
3,7,2023-02-01,B,C,B,C,B,C
4,10,2023-02-01,None,None,None,None,None,None


Assim como na criação de variáveis numéricas, podemos criar um arquivo no formato .sql e executar a query direto no banco de dados. Segue um exemplo:

In [ ]:
#|eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1', 'FEAT_CAT2']
lista_janela = [1, 3, 6]
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query_final_cat_mysql = mysql_create_query_cat(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=connection)

Complete query creation with table.sql saved file


Verificando tabela criada:

In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, connection)
df.head()

/tmp/ipykernel_10461/3805114983.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M,FEAT_CAT1_MODA_6M,FEAT_CAT2_MODA_6M
0,4,2023-02-01,None,None,None,None,None,None
1,5,2023-02-01,A,A,A,A,A,A
2,6,2023-02-01,None,None,None,None,None,None
3,7,2023-02-01,B,C,B,C,B,C
4,10,2023-02-01,None,None,None,None,None,None


### Criação de variáveis agregadas

Podemos criar variáveis agregando por valor específico de variável categórica e calculando as operações de sum, avg, min e max para as variáveis númericas definidas pelo usuário. Por exemplo, para a variável ``FEAT_CAT1`` tendo valor igual a ``A`` ou ``B``, calculamos as operações de sum, avg, min e max para as variáveis ``FEAT_NUM1`` e ``FEAT_NUM2``.  Segue um caso de uso:

In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = mysql_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_mysql_agregada = pd.read_sql(query, connection)
df_mysql_agregada.head()

/tmp/ipykernel_10461/789764728.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql_agregada = pd.read_sql(query, connection)


,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT2'
lista_valor_agragador = ['B', 'C']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = mysql_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

Complete query creation with no saved file.


In [ ]:
#|eval: false

df_mysql_agregada = pd.read_sql(query, connection)
df_mysql_agregada.head()

/tmp/ipykernel_10461/789764728.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql_agregada = pd.read_sql(query, connection)


,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT2_B_3M,MAX_FEAT_NUM1_FEAT_CAT2_B_3M,MIN_FEAT_NUM1_FEAT_CAT2_B_3M,AVG_FEAT_NUM1_FEAT_CAT2_B_3M,SUM_FEAT_NUM2_FEAT_CAT2_B_3M,MAX_FEAT_NUM2_FEAT_CAT2_B_3M,MIN_FEAT_NUM2_FEAT_CAT2_B_3M,AVG_FEAT_NUM2_FEAT_CAT2_B_3M,SUM_FEAT_NUM1_FEAT_CAT2_B_6M,MAX_FEAT_NUM1_FEAT_CAT2_B_6M,MIN_FEAT_NUM1_FEAT_CAT2_B_6M,AVG_FEAT_NUM1_FEAT_CAT2_B_6M,SUM_FEAT_NUM2_FEAT_CAT2_B_6M,MAX_FEAT_NUM2_FEAT_CAT2_B_6M,MIN_FEAT_NUM2_FEAT_CAT2_B_6M,AVG_FEAT_NUM2_FEAT_CAT2_B_6M,SUM_FEAT_NUM1_FEAT_CAT2_C_3M,MAX_FEAT_NUM1_FEAT_CAT2_C_3M,MIN_FEAT_NUM1_FEAT_CAT2_C_3M,AVG_FEAT_NUM1_FEAT_CAT2_C_3M,SUM_FEAT_NUM2_FEAT_CAT2_C_3M,MAX_FEAT_NUM2_FEAT_CAT2_C_3M,MIN_FEAT_NUM2_FEAT_CAT2_C_3M,AVG_FEAT_NUM2_FEAT_CAT2_C_3M,SUM_FEAT_NUM1_FEAT_CAT2_C_6M,MAX_FEAT_NUM1_FEAT_CAT2_C_6M,MIN_FEAT_NUM1_FEAT_CAT2_C_6M,AVG_FEAT_NUM1_FEAT_CAT2_C_6M,SUM_FEAT_NUM2_FEAT_CAT2_C_6M,MAX_FEAT_NUM2_FEAT_CAT2_C_6M,MIN_FEAT_NUM2_FEAT_CAT2_C_6M,AVG_FEAT_NUM2_FEAT_CAT2_C_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Segue um exemplo para salvar um arquivo com a query criada e criar a tabela diretamente no banco de dados:

In [ ]:
#|eval: false

tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'
status = True
table_name = 'nome_da_tabela_criada'
nome_arquivo = 'table.sql'

query = mysql_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador, nome_arquivo=nome_arquivo, status=status, table_name=table_name, conn=connection)

Complete query creation with table.sql saved file


Para conferir a criação, vamos consultar o banco de dados com o nome da tabela que foi criada:

In [ ]:
# |eval: false

query=f"""SELECT  * from {table_name}"""
df = pd.read_sql(query, connection)
df.head()

/tmp/ipykernel_10461/3805114983.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Snowflake

### Criação de variáveis numéricas

A função snow_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]

query_final_num_snow = snow_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
print(query_final_num_snow)


    

    WITH 
    tb_public AS (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M AS (
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MAX_1M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_AVG_1M,
            MEDIAN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MEDIAN_1M,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,

Podemos salvar a consulta da query em um arquivo também:

In [ ]:
# |eval: false

tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
nome_arquivo = 'table.sql'

query_final_num_snow = snow_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra, nome_arquivo)

Complete query creation with table.sql saved file


### Criação de variáveis categóricas

A função query_final_cat_snow() cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1','FEAT_CAT2']
lista_janela = [1,2,3]

query_final_cat_snow = snow_create_query_cat(tb_publico, tb_feat, lista_janela, feat_num_lista, id, safra_ref, safra)

Complete query creation with no saved file.


In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1','FEAT_CAT2']
lista_janela = [1,2,3]
nome_arquivo = 'table.sql'

query_final_cat_snow = snow_create_query_cat(tb_publico, tb_feat, lista_janela, feat_num_lista, id, safra_ref, safra, nome_arquivo)

Complete query creation with table.sql saved file


### Criação de variáveis agregadas

Podemos criar variáveis agregando por valor específico de variável categórica e calculando as operações de sum, avg, min e max para as variáveis númericas definidas pelo usuário. Por exemplo, para a variável ``FEAT_CAT1`` tendo valor igual a ``A`` ou ``B``, calculamos as operações de sum, avg, min e max para as variáveis ``FEAT_NUM1`` e ``FEAT_NUM2``.  Segue um caso de uso:

In [ ]:
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['B', 'C']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'
janelas = [1, 2, 3]
tb_publico = 'tb_spine'


query_final_cat_agre_snow = snow_create_query_agregada(tb_publico, tb_feat, janelas,  lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

Complete query creation with no saved file.


Podemos também salvar a consulta em um arquivo:

In [ ]:
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['B', 'C']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'
janelas = [1, 2, 3]
tb_publico = 'tb_spine'
nome_arquivo = 'table.sql'

query_final_cat_agre_snow = snow_create_query_agregada(tb_publico, tb_feat, janelas,  lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador, nome_arquivo)

Complete query creation with table.sql saved file
